In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13881245107498236156
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4185718784
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16208077052002835652
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
market_data = pd.read_csv("./data/Bitcoin Price Weekly 2022 to 2010 .csv")
graph_parameters= pd.read_csv("./data/weeklyparameters.csv")
scaler = StandardScaler()
graph_parameters = pd.DataFrame(scaler.fit_transform(graph_parameters))
market = market_data.iloc[50:50+graph_parameters.shape[0],1:].reset_index().drop('index',axis=1)

In [4]:
full_df= pd.concat([market,graph_parameters],axis=1).iloc[:116]

In [5]:
full_df

,Price,Open,High,Low,Vol.,Change %,0,1,2,3,...,246,247,248,249,250,251,252,253,254,255
0,14.4,15.4,16.5,11.0,366200.0,-6.62,1.336229,1.939600,0.898095,1.046359,...,1.857811,1.640449,1.959683,1.505424,1.262469,0.679864,1.392436,1.902967,1.171726,1.441735
1,13.7,14.4,15.7,13.5,190880.0,-4.59,0.242123,-0.345267,0.441242,-2.459430,...,1.125491,0.877204,1.271590,0.999982,1.462776,0.826972,1.268366,1.078491,1.021015,1.040968
2,13.7,13.7,14.7,12.5,263670.0,0.00,0.171934,1.556844,-0.167209,-0.130687,...,0.956016,1.375935,1.168721,0.883136,1.095990,0.878241,0.996945,1.018847,0.838051,1.130266
3,13.5,13.7,14.7,13.3,132600.0,-1.10,-1.667505,0.961015,-1.847863,-0.317615,...,-0.754241,-0.916805,-1.233785,-1.140221,-0.658978,-0.587275,-0.778278,-0.906391,-1.771800,-1.261268
4,6.6,13.5,14.9,5.7,398270.0,-51.59,-0.624569,1.721534,-0.747817,-2.203964,...,-2.531938,-2.090335,-2.303107,-2.410805,-2.665507,-2.574860,-2.405070,-2.553615,-2.582007,-2.112554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,112.8,103.0,115.0,102.7,139760.0,9.47,0.661311,-0.153109,0.651302,1.306532,...,0.109536,0.782358,1.367012,0.704425,0.402328,0.730888,0.444774,0.297567,0.059070,0.017453
111,119.6,112.8,125.0,111.8,160000.0,6.08,0.531979,-0.825681,1.229333,0.947948,...,-0.106460,0.604899,0.811877,0.729993,0.046917,0.093446,0.365070,0.486385,0.199162,0.532466
112,141.0,119.6,148.7,119.1,170740.0,17.89,-1.563187,0.687899,1.894061,-0.687893,...,-1.186053,-0.798745,-0.874481,-1.772869,-1.274347,-1.247473,-1.195304,-1.239829,-0.181167,-1.014861
113,129.0,141.0,148.9,121.3,130890.0,-8.52,0.930595,1.583702,1.560291,-0.609328,...,0.398205,0.472617,0.275036,-0.015382,0.099145,0.134643,0.243416,-0.482003,0.191174,-0.104870


In [6]:
threshold=3
def y_creator(threshold):
    Y = []
    for i in market_data["Change %"][51:51+graph_parameters.shape[0]]:
        if abs(i)>threshold:
            if i>0:
                Y.append(1)
            else:
                Y.append(-1)
        else:
            Y.append(0)
    return Y
Y = y_creator(threshold)